# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark, pyspark
findspark.find()

'C:\\bigdata\\spark-2.4.5-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master('local').appName('cust_consult').getOrCreate()

In [3]:
data = spark.read.csv('data/customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
], outputCol='features')

In [9]:
output = assembler.transform(data)

In [11]:
final_data = output.select('features', 'churn')
final_data.count()

900

In [12]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [13]:
train_churn.count()

627

In [14]:
test_churn.count()

273

In [15]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')

In [16]:
fitted_churn_model = lr_churn.fit(train_churn)

In [17]:
training_sum = fitted_churn_model.summary

In [19]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                627|                627|
|   mean|0.17065390749601275|0.12759170653907495|
| stddev| 0.3765066260420354| 0.3339009999364852|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [22]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.85434454709609...|[0.70147771549448...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.71247183735151...|[0.99878582051892...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.10864064698842...|[0.99969916286800...|       0.0|
|[28.0,11245.38,0....|    0|[3.89193518463381...|[0.98000225165168...|       0.0|
|[29.0,10203.18,1....|    0|[3.88194930059243...|[0.97980560898710...|       0.0|
|[29.0,13255.05,1....|    0|[4.15830398115187...|[0.98460660995396...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.84934648982148...|[0.97915031826574...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.27513368724504...|[0.98628064977453...|       0.0|
|[30.0,12788.37,0....|    0|[2.60548349634970...|[0.93121365544896...|       0.0|
|[31.0,10182.6,1

In [23]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [24]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [25]:
auc

0.7849342770475227

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', 
                                              labelCol='churn',
                                              metricName='accuracy')

In [30]:
acc = evaluator.evaluate(pred_and_labels.predictions)

In [31]:
acc

0.9084249084249084

In [32]:
final_lr_model = fitted_churn_model

In [33]:
new_customers = spark.read.csv('./data/new_customers.csv', inferSchema=True, header=True)

In [34]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [35]:
test_new_customers = assembler.transform(new_customers)

In [36]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
final_results = final_lr_model.transform(test_new_customers)

In [38]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

